# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from A3C_Agent import *
from ENV_DETAILS import *

2023-12-03 10:30:33.542758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 10:30:33.542780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 10:30:33.543365: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
ENV = "MountainCarContinuous-v0"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [6]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<PendulumEnv<Pendulum-v1>>>>

In [7]:
env._max_episode_steps

200

In [4]:
s = env.observation_space.sample()
s

array([-0.03835609, -0.05998918], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), (2,))

In [6]:
s = env.reset()
s[0]

array([-0.5637154,  0.       ], dtype=float32)

In [7]:
s = env.reset()[0]
env.action_space, env.action_space, env.step(s)

(Box(-1.0, 1.0, (1,), float32),
 Box(-1.0, 1.0, (1,), float32),
 (array([-0.495756  , -0.00095793], dtype=float32),
  -0.02448251243676083,
  False,
  False,
  {}))

# Advantage Asynch Actor-Critic 

This time we will implement the A3C not considering Actor and Critic as part of the same network

### Training/Hyperparam run

In [8]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_A3C/"

if TUNING_TYPE == "MANUAL":
    params = {}
    params["n_enviroment"] =[20]
    params["discount"] =[0.96, 0.97]
    params["end_of_episode"] = [400]
    params["learning_rate"] = [0.001]
    params["entropy_factor"] = [ 0.01, 0.05]
    params["epsilon"] = [1]
    params["boltzman_factor"] = [1]
    params["exploration_technique"] = ['soft', 'epsilon']
    params["training_steps"] = [3000000]
    params["dense_units"] = [32, 128]
    params["time_to_update"] = [400]
    params["use_LSTM"] =[False]


    hyperparam_combination = list(itertools.product(*list(params.values())))
    

    try:

        files = [name for name in os.listdir(logs_dir) if os.path.isfile(os.path.join(logs_dir, name)) and name != "logfile.txt" and name != "merged_results.json"]
        if len(files) >= 1: merge_JsonFiles(main_hyper_dir, logs_dir, files)

        res_file = logs_dir+"merged_results.json"
        def without_keys(d, keys):
            return {x: d[x] for x in d if x not in keys}


        if os.path.isfile(res_file):
            with open(res_file, 'r') as f:
                complete_file = json.load(f)

            newlist = sorted(complete_file, key=lambda d: d['mean_rewards'], reverse=True) 
            params = []
            for i, f in enumerate(newlist):
                label = "disc : "+str(f['discount'])+" | "+"lr : "+str(f['learning_rate'])+" | "+"entropy : "+str(f['entropy_factor'])+" | "+"update : "+str(f['time_to_update'])
                plt.figure(figsize=[35,4])
                plt.plot(f['rewards'], label = label)
                plt.legend()
                max_mean_reward = f['mean_rewards']
                params.append(f)
                print(without_keys(f,"rewards"))
                if i == 10:
                    break
            plt.title("Evaluation rewards"); plt.grid()
            plt.show()

    except:
        pass



In [10]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_A3C"
    project_name = "keras_tunning_soft_continuous"

    tuner = kt.BayesianOptimization(
            MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper_continuous/A3C_soft/", 
                          end_of_episode = EPISODES, n_enviroment = 10, 
                  evaluation_epoch = 2000, training_steps = 300000,
                  sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                  discount_min = 0.8, discount_max = 0.99,
                  entropy_min = 0.000001, entropy_max = 0.05,
                  lr_min = 0.000001, lr_max = 0.005,
                  dense_min = 32, dense_max = 500,
                  lstm_min = 32, lstm_max = 256,
                  time_to_update_min = 100, time_to_update_max=600,
                  environment_name=ENV,
                  continuous_actions_space= True,
                  reward_scaler = 1,
                  n_dense_layers = 4,
                  use_LSTM=False
                  ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 30,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
    
   
else : 
    
        print("Acquiring parameters ....")
        writer= "Training/fit_A3C/"

        n_enviroment = 10
        training_steps = 1000000
        learning_rate= 0.001
        entropy_factor = 0.1
        discount = 0.99
        dense_units = 512
        lstm_units = 128
        time_to_update= 100
        end_of_episode = 300
        use_LSTM = False

        
        run_training(training_steps, learning_rate, entropy_factor, "", discount, -1, dense_units, lstm_units, n_enviroment, writer, use_LSTM, end_of_episode,
                      environment_name = ENV,continuous_space_actions = True, sigma_noise = 1e-5, reward_scaler = 1)
        
        
            

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_soft_continuous/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [11]:

exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    n_env = 30
    end_ep = 1000
    ep = 1
    bolt_fact = 1
    train_steps = 5000000
    disc = trials.hyperparameters.values["discount"]
    lr = trials.hyperparameters.values["learning_rate"]
    entropy_fact = trials.hyperparameters.values["entropy_factor"]
    n_hidden_layer_hp = trials.hyperparameters.values["n_dense_layers"]

    d = []
    for i in range(n_hidden_layer_hp):
        d.append(trials.hyperparameters.values['dense_units_'+str(i)])
    ulstm = False

    # hyperparam_combination.append((n_env, disc, end_ep, lr, entropy_fact, ep, bolt_fact, exploration_tech, train_steps, d, time_to_update, ulstm))

Trial id :28 | Score :84.28267448105629 --> {'discount': 0.99, 'entropy_factor': 1e-05, 'learning_rate': 0.005, 'n_dense_layers': 1, 'dense_units_0': 364, 'dense_units_1': 500, 'dense_units_2': 32, 'dense_units_3': 500}
Trial id :27 | Score :-0.005297040558535487 --> {'discount': 0.99, 'entropy_factor': 1e-05, 'learning_rate': 0.0006738217144122209, 'n_dense_layers': 1, 'dense_units_0': 208, 'dense_units_1': 183, 'dense_units_2': 32, 'dense_units_3': 319}
Trial id :24 | Score :-0.005396947087733275 --> {'discount': 0.97, 'entropy_factor': 1e-05, 'learning_rate': 1e-05, 'n_dense_layers': 3, 'dense_units_0': 305, 'dense_units_1': 158, 'dense_units_2': 369, 'dense_units_3': 258}


In [12]:
len(hyperparam_combination), hyperparam_combination

(0, [])

In [13]:
# run_hyperparam(TUNING_TYPE= "MANUAL", TUNING_TYPE = TUNING_TYPE, hyperparam_combination = hyperparam_combination,  total_files = 4)

In [14]:

val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_A3C"

exploration_tech ="soft"

for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
    

    env_model = tuner.get_best_models()[0]
    n_enviroment = 30
    end_ep = 1000
    ep = 1
    bolt_fact = 1
    training_steps = 2000000
    discount = trials.hyperparameters.values["discount"]
    learning_rate = trials.hyperparameters.values["learning_rate"]
    entropy_factor = trials.hyperparameters.values["entropy_factor"]
    n_hidden_layer_hp = trials.hyperparameters.values["n_dense_layers"]

    dense_units = []
    for i in range(n_hidden_layer_hp):
        dense_units.append(trials.hyperparameters.values['dense_units_'+str(i)])
    sigma_noise = 1e-5
    time_to_update= 100
    lstm_units= 32
    use_LSTM=False
    end_of_episode = EPISODES
    save_factor=50000
    sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
    sucess_criteria_value = SUCESS_CRITERIA_VALUE
    environment_name = ENV
    reward_scaler = 16
    continuous_space_actions = True
    return_agent = True

# env_model = run_training(training_steps, learning_rate, entropy_factor, exploration_tech, discount, time_to_update, dense_units, lstm_units, 
#                  n_enviroment, writer, use_LSTM, end_of_episode, save_factor, sucess_criteria_epochs , sucess_criteria_value , 
#                  environment_name, reward_scaler, continuous_space_actions, sigma_noise, return_agent)

# env_model = ContinuousA3CAgent_Optimization(
#             lr = learning_rate, 
#             entropy_factor = entropy_factor,
#             discount = discount, 
#             dense_units= dense_units, 
#             lstm_units = lstm_units,
#             n_enviroment = n_enviroment,
#             writer = writer,
#             trial_n = get_valid_trials_number(writer),
#             use_LSTM=use_LSTM,
#             end_of_episode = end_of_episode,
#             environment_name = environment_name,
#             sigma_noise= sigma_noise,
#             reward_scaler = reward_scaler,
#             use_GAE = False
#         )
# env_model.load_weights('./checkpoints/A2Cagent')
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./A3C_continuous_"+exploration_tech+"_video.mp4", continuous_action_space = continuous_space_actions)
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))



Trial id :28 | Score :84.28267448105629 --> {'discount': 0.99, 'entropy_factor': 1e-05, 'learning_rate': 0.005, 'n_dense_layers': 1, 'dense_units_0': 364, 'dense_units_1': 500, 'dense_units_2': 32, 'dense_units_3': 500}
Trial number :  30
Moviepy - Building video ./A3C_continuous_soft_video.mp4.
Moviepy - Writing video ./A3C_continuous_soft_video.mp4



Moviepy - Done !
Moviepy - video ready ./A3C_continuous_soft_video.mp4
